# In this notebook I will:
# - read MLD climatology for chosen region
# - interpolate depth profiles for WOA temp data
# - average all temp data above MLD: GET SST FORCING

go!

## 1. of all: read MLD climatology

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob

In [2]:
mld1 = xr.open_dataset('MLDClimatology_DeBoyerMontagut/mld_DT02_c1m_reg2.0.nc')

/Users/bpo/anaconda/anaconda3/lib/python3.7/site-packages/xarray/coding/times.py:465: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/Users/bpo/anaconda/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


In [3]:
mld1

<xarray.Dataset>
Dimensions:       (lat: 90, lon: 180, time: 12)
Coordinates:
  * lat           (lat) float32 -88.0 -86.0 -84.0 -82.0 ... 84.0 86.0 88.0 89.5
  * lon           (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time          (time) object 0001-01-15 12:00:00 ... 0001-12-15 12:00:00
Data variables:
    krig_std_dev  (time, lat, lon) float32 ...
    mask          (lat, lon) float32 ...
    med_dev       (time, lat, lon) float32 ...
    mld           (time, lat, lon) float32 ...
    mld_raw       (time, lat, lon) float32 ...
    mld_smth      (time, lat, lon) float32 ...
    n_profiles    (time, lat, lon) float32 ...
Attributes:
    File_Name:              mld_T02_ALL_clim_1941_2008.nc
    Producer_Agency:        IFREMER - LOS - Tue Nov  4 22:11:08 2008
    Methodology:            de Boyer Montegut et al., JGR 2004 (http://www.lo...
    Associate_File:         06_test_bl_thadathil_as/B_PREGRID/PREGRID_ALL_mld...
    Source_Files:           WOCE Global Data V

In [4]:
def MLD_at_latlon(lats=40,lons=180-20, mlddata=mld1, justMLD=True):
    """This function returns MLD depth across the year at a specific location"""
    #ToDO include check whether the spot is on land or water
    #Make sure the lat lon usage across everything is coherent!
    
    mld2 = mlddata.sel(lat=lats,lon=lons,method='nearest').to_dataframe()
    mld2['time'] = mld2.index
    mld2.index = range(1,13)
    mld2.index.names = ['time']
    
    if justMLD==True:
        return mld2[['mld']]#.reset_index()
    else:
        return mld2#.reset_index()
    
#MLD_at_latlon()
MLD4020 = MLD_at_latlon().reset_index()

In [5]:
TempNCDFfiles = glob.glob('WOA2018/Temperature/*.nc', recursive=True)

WOAtemp = xr.open_mfdataset(TempNCDFfiles,combine='by_coords',decode_times=False)

In [6]:
len([i for j in range(0,57) for i in range(1,13)])
#len(WOAtemp_df.index.get_level_values(level='depth'))

684

In [7]:
def WOATempInLocation(lats=40,lons=-20, WOAtempdata=WOAtemp, justTempDepth=True):
    """This function returns the xarray of Temperature in a specific location"""
    WOAtemp1 = WOAtempdata.sel(nbounds=0)
    WOAtemp_df = WOAtemp1.sel(lat=lats,lon=lons, method='nearest').to_dataframe()
    WOAtemp_df.index = pd.MultiIndex.from_arrays([WOAtemp_df.index.get_level_values(level='depth'), [i for j in range(0,57) for i in range(1,13)]])
    WOAtemp_df.index.names = ['depth','time']
    
    if justTempDepth==True:
        return WOAtemp_df[['t_an']]#.reset_index()
    else:
        return WOAtemp_df#.reset_index()

    
WOAtempprofiles = WOATempInLocation()

In [8]:
print(WOAtempprofiles.head())
MLD4020.head()

                t_an
depth time          
0.0   1     14.87860
      2     14.39471
      3     14.18719
      4     14.59340
      5     15.61929


,time,mld
0,1,119.166565
1,2,131.147095
2,3,106.412193
3,4,58.556564
4,5,32.214375


In [9]:
def interpolateWOAprofiles(WOAdat=WOAtempprofiles):
    """this function interpolates the WOA profiles to a resolution of 1 meter"""
    WOAgrid = WOAdat.unstack(level=1)
    WOAgrid1 = WOAgrid.reindex(range(0,1501))
    return WOAgrid1.interpolate(method='linear')
    
    
WOAtemp_int = interpolateWOAprofiles()

In [10]:
def MeanTempAboveMLD(WOAint=WOAtemp_int, MLD=MLD4020):
    """this function filters all temperature measurements above a certain depth and takes the mean"""
    WOAint_d1 = WOAint
    WOAint_d1.columns = WOAint.columns.droplevel()
    WOAx = WOAint_d1.reset_index()
    out = []
    for i in range(1,13):
        out.append({'Month':i,'Temp':WOAx[WOAx['depth'] < MLD[MLD['time']==i]['mld'].values[0]][i].mean()})
    return pd.DataFrame(out)
        
TempAboveMLD1 = MeanTempAboveMLD()

In [11]:
TempAboveMLD1

,Month,Temp
0,1,14.704305
1,2,14.200098
2,3,14.047897
3,4,14.423152
4,5,15.369067
5,6,17.453592
6,7,19.781525
7,8,21.022924
8,9,20.917368
9,10,19.527817


In [11]:
WOAtemp_int_d1 = WOAtemp_int
WOAtemp_int_d1.columns = WOAtemp_int.columns.droplevel()
WOAtemp_int_dep = WOAtemp_int_d1.reset_index()

In [36]:
WOAtemp_int_dep['depth'] < MLD4020[MLD4020['time']==1]['mld'].values[0]

0        True
1        True
2        True
3        True
4        True
        ...  
1496    False
1497    False
1498    False
1499    False
1500    False
Name: depth, Length: 1501, dtype: bool

In [41]:
for i in range(1,13):
    WOAtemp_int_dep[WOAtemp_int_dep['depth'] < MLD4020[MLD4020['time']==i]['mld'].values[0]][i].mean()
    

14.704304695129395 119.166565
14.200098037719727 131.1471
14.047897338867188 106.41219
14.423151969909668 58.556564
15.369067192077637 32.214375
17.45359230041504 20.535313
19.781524658203125 15.43875
21.022924423217773 14.662803
20.917367935180664 25.207499
19.527816772460938 40.113438
17.48056983947754 64.45242
16.03085708618164 78.88098


In [288]:
WOAtemp_int[1].index #.loc[(WOAtemp_int.index < 5)]

WOAtemp_int[1].index < MLD4020[MLD4020['time']==1]['mld']

ValueError: Lengths must match to compare

In [284]:
for i in range(1,13):
    print(WOAtemp_int[i].index > MLD4020[MLD4020['time']==i]['mld'])
    

ValueError: Lengths must match to compare

In [261]:
WOAtemp_int.index['depth']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [184]:
#WOAtempprofiles.index.levels[-1].astype(int)

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64', name='time')

In [233]:
WOAgridZ = WOAtempprofiles.unstack(level=1)

WOAgridX = WOAgridZ.reindex(range(0,1501))

WOAgridX.interpolate(method='linear')

t_an                                                         \
time          1          2          3          4          5          6    
depth                                                                     
0      14.878600  14.394710  14.187190  14.593400  15.619290  17.717110   
1      14.880698  14.391625  14.192914  14.591838  15.605394  17.705166   
2      14.882796  14.388542  14.198638  14.590276  15.591498  17.693222   
3      14.884894  14.385458  14.204362  14.588715  15.577601  17.681278   
4      14.886992  14.382375  14.210086  14.587152  15.563705  17.669334   
...          ...        ...        ...        ...        ...        ...   
1496    5.716068   5.508822   5.334196   5.448516   5.538819   5.470336   
1497    5.707529   5.499244   5.329419   5.441465   5.530990   5.465602   
1498    5.698989   5.489666   5.324643   5.434413   5.523160   5.460868   
1499    5.690450   5.480088   5.319867   5.427361   5.515330   5.456134   
1500    5.681910   5.470510   5.315090   5.420310   5.507500   5.451400   

                                                                         
time          7          8          9          10         11         12  
depth                                                                    
0      20.028090  21.151699  21.050510  19.678909  17.649200  16.124300  
1      20.000914  21.133541  21.040531  19.669865  17.650000  16.122520  
2      19.973738  21.115383  21.030550  19.660822  17.650801  16.120741  
3      19.946562  21.097225  21.020571  19.651777  17.651600  16.118959  
4      19.919386  21.079067  21.010590  19.642735  17.652401  16.117180  
...          ...        ...        ...        ...        ...        ...  
1496    5.583497   5.494810   5.434471   5.435601   5.580342   5.448673  
1497    5.577525   5.488310   5.426804   5.428953   5.573034   5.437607  
1498    5.571554   5.481810   5.419136   5.422306   5.565726   5.426541  
1499    5.565582   5.475310   5.411468   5.415658   5.558418   5.415476  
1500    5.559610   5.468810   5.403800   5.409010   5.551110   5.404410  

[1501 rows x 12 columns]

In [216]:
WOAtempprofiles.groupby('time').transform(pd.DataFrame.interpolate)

,depth,t_an
0,0.0,14.87860
1,0.0,14.39471
2,0.0,14.18719
3,0.0,14.59340
4,0.0,15.61929
...,...,...
679,1500.0,5.46881
680,1500.0,5.40380
681,1500.0,5.40901
682,1500.0,5.55111


In [73]:
#ds2.variables
ds3 = ds2.sel(nbounds=0, time=371.5+1)

df = ds3.sel(lat=40,lon=-20, method='nearest').to_dataframe()

df

lat   lon         crs  lat_bnds  lon_bnds  depth_bnds  \
depth  time                                                            
0.0    372.5  40.5 -19.5 -2147483647      40.0     -20.0         0.0   
       373.5  40.5 -19.5 -2147483647      40.0     -20.0         0.0   
       374.5  40.5 -19.5 -2147483647      40.0     -20.0         0.0   
       375.5  40.5 -19.5 -2147483647      40.0     -20.0         0.0   
       376.5  40.5 -19.5 -2147483647      40.0     -20.0         0.0   
...            ...   ...         ...       ...       ...         ...   
1500.0 379.5  40.5 -19.5 -2147483647      40.0     -20.0      1475.0   
       380.5  40.5 -19.5 -2147483647      40.0     -20.0      1475.0   
       381.5  40.5 -19.5 -2147483647      40.0     -20.0      1475.0   
       382.5  40.5 -19.5 -2147483647      40.0     -20.0      1475.0   
       383.5  40.5 -19.5 -2147483647      40.0     -20.0      1475.0   

              climatology_bounds      t_an       t_mn  t_dd      t_sd  \
depth  time                                                             
0.0    372.5                 0.0  14.87860  14.765243   8.0  0.403968   
       373.5                 1.0  14.39471  14.585494  23.0  0.552522   
       374.5                 2.0  14.18719  14.013444  18.0  0.814553   
       375.5                 3.0  14.59340  14.430242  13.0  0.645615   
       376.5                 4.0  15.61929  15.573319  15.0  0.987385   
...                          ...       ...        ...   ...       ...   
1500.0 379.5                 7.0   5.46881   5.303387   7.0  0.305868   
       380.5                 8.0   5.40380   5.493583  13.0  0.453310   
       381.5                 9.0   5.40901   5.163136   5.0  0.434501   
       382.5                10.0   5.55111   5.697942   5.0  0.495621   
       383.5                11.0   5.40441   5.751211   3.0  0.359141   

                  t_se      t_oa      t_ma  t_gp  
depth  time                                       
0.0    372.5  0.142824 -0.113358 -2.377490  67.0  
       373.5  0.115209  0.190784 -2.861381  67.0  
       374.5  0.191992 -0.173746 -3.068900  67.0  
       375.5  0.179061 -0.163158 -2.662690  67.0  
       376.5  0.254942 -0.045971 -1.636800  67.0  
...                ...       ...       ...   ...  
1500.0 379.5  0.115607 -0.165423 -0.001480  67.0  
       380.5  0.125726  0.089783 -0.066490  67.0  
       381.5  0.194315 -0.245874 -0.061280  66.0  
       382.5  0.221648  0.146832  0.080820  67.0  
       383.5  0.207350  0.346801 -0.065880  66.0  

[684 rows x 15 columns]